<a href="https://colab.research.google.com/github/rpatel71/Design-Optimization/blob/main/Homework_4/HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.linalg import inv

In [ ]:
fun = lambda x: (x[0] ** 2 + x[1] ** 2 + x[2] ** 2)   # Objective function
constraint_1 = lambda x: (x[0] ** 2 / 2 + x[1] ** 2 / 5 + x[2] ** 2 / 25 - 1)   # Constraint 1 from the given constraints
constraint_2 = lambda x: (x[0] + x[1] - x[2])   # Constraint 2 from the given constaints

h = np.array([cons1, cons2]   # h is an array of constraints